In [ ]:
import sys
sys.path.append('../') 
%load_ext autoreload
%autoreload 2
import sklearn
import copy
import numpy as np

import seaborn as sns
sns.set()

import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
# from viz import viz
from bokeh.plotting import figure, show, output_notebook, output_file, save
from functions import merge_data
from sklearn.model_selection import RandomizedSearchCV
import load_data
import exponential_modeling


from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import fit_and_predict

In [ ]:
df = load_data.load_county_level(data_dir = '../data/')
#df = df.sort_values('#Deaths_4/3/20', ascending=False)
# outcome_cases = load_data.outcome_cases # most recent day
# outcome_deaths = load_data.outcome_deaths
important_vars = load_data.important_keys(df)
very_important_vars = ['PopulationDensityperSqMile2010',
#                        'MedicareEnrollment,AgedTot2017',
                       'PopulationEstimate2018',
                       '#ICU_beds',
                       'MedianAge2010',
                       'Smokers_Percentage',
                       'DiabetesPercentage',
                       'HeartDiseaseMortality',
                        '#Hospitals'
#                        'PopMale60-642010',
#                         'PopFmle60-642010',
#                          'PopMale65-742010',
#                          'PopFmle65-742010',
#                          'PopMale75-842010',
#                          'PopFmle75-842010',
#                          'PopMale>842010',
#                          'PopFmle>842010'
                      ]

In [ ]:
max_cases = [max(v) for v in df['cases']]
df['max_cases'] = max_cases
df =  df[df['max_cases'] > 0]

In [ ]:
exponential = {'model_type':'exponential'}
shared_exponential = {'model_type':'shared_exponential'}
demographics = {'model_type':'shared_exponential', 'demographic_vars':very_important_vars}
linear = {'model_type':'linear'}

## predictions we save: four separate models, simple ensemble model with exponential and shared_exponential , ensemble with all possible combinations of 3 models, full ensemble

In [ ]:
all_methods = [exponential, 
               shared_exponential,
               demographics,
               linear]
df = fit_and_predict.fit_and_predict_ensemble(df, 
                                              target_day=np.array(range(1, 22)),
                                              mode='predict_future',
                                              outcome='deaths',
                                              methods=all_methods,
                                              output_key=f'predicted_deaths_ensemble_all'
                                              )
df = fit_and_predict.fit_and_predict_ensemble(df, 
                                              target_day=np.array(range(1, 22)),
                                              mode='predict_future',
                                              outcome='deaths',
                                              methods=all_methods[:2],
                                              output_key=f'predicted_deaths_simple_ensemble_all'
                                              )
for (i, model) in enumerate(all_methods):
        
    if 'demographic_vars' in model:
        demographic_vars = model['demographic_vars']
    else:
        demographic_vars = []
    
    method = model['model_type']
    #print(method)
    method_name = model['model_type']
    if 'demographic_vars' in model:
        method_name = 'demographics'
    df = fit_and_predict.fit_and_predict(df, 
                                         outcome='deaths', 
                                         method=method, 
                                         mode='predict_future', 
                                         target_day=np.array(range(1, 22)),
                                         output_key=f'predicted_deaths_{method_name}_all',
                                         demographic_vars=demographic_vars)
    print(all_methods[0:i]+all_methods[(i+1):])
    df = fit_and_predict.fit_and_predict_ensemble(df, 
                                              target_day=np.array(range(1, 22)),
                                              mode='predict_future',
                                              outcome='deaths',
                                              methods=all_methods[0:i]+all_methods[(i+1):],
                                              output_key=f'predicted_deaths_ensemble_no_{method_name}_all'
                                              )

In [ ]:
method_keys = [c for c in df if 'predicted' in c]

In [ ]:
for key in method_keys:
    for d in range(1, 8):
        newkey = key[:-3] + str(d)
        df[newkey] = np.array([p[d-1] for p in df[key].values])

In [ ]:
df['predicted_deaths_demographics_7']

In [ ]:
[c for c in df if 'predicted' in c]

In [ ]:
for c in df:
    if 'predicted' in c and 'all' not in c:
        print((c, df[c].max()))

In [ ]:
method_keys = [c for c in df if 'predicted' in c]
geo = ['countyFIPS', 'CountyNamew/StateAbbrev']
preds_df = df[method_keys + geo]
preds_df.to_csv("../predictions/predictions_04_03.csv")

In [ ]:
df['deaths'].values[0]

## State-level predictions

In [445]:
aggregate_by_state = True

In [446]:
def sum_lists(list_of_lists):
    arr = np.array(list(list_of_lists))
    sum_arr = np.sum(arr,0)
    return list(sum_arr)

def get_90_quantile(feature):
    return np.quantile(np.array(feature), .9)

In [447]:
from os.path import join as oj
def load_state_data(usafacts_data_cases='usafacts/confirmed_cases.csv',
                    usafacts_data_deaths='usafacts/deaths.csv',
                    dir_mod = ""):
    
    """
    load state-level data.
    
    For cases/deaths data, this is almost the same as load_usafacts, with unallocated deaths added to each state
    For demographic variables, data are aggregated from county level
    
    
    """
    usafacts_data_cases = oj(dir_mod, usafacts_data_cases)
    usafacts_data_deaths = oj(dir_mod, usafacts_data_deaths)
    
    cases = pd.read_csv(usafacts_data_cases, encoding="iso-8859-1", index_col=0).T
    deaths = pd.read_csv(usafacts_data_deaths, encoding="iso-8859-1", index_col=0).T
    if not 'countyFIPS' in deaths.keys():
        deaths = pd.read_csv(usafacts_data_deaths, encoding="utf-8", index_col=0).T
    # change to int type
    for col in cases.columns:
        if not 'county' in col.lower() and not 'state' in col.lower():
            cases[col] = cases[col].astype(float).astype(int)
            deaths[col] = deaths[col].astype(float).astype(int)
  
 
    cases = cases.rename(columns={k: '#Cases_' + k for k in cases.keys()
                                  if not 'county' in k.lower()
                                  and not 'state' in k.lower()})


    deaths = deaths.rename(columns={k: '#Deaths_' + k for k in deaths.keys()
                                    if not 'county' in k.lower()
                                    and not 'state' in k.lower()})

    deaths.countyFIPS = deaths.countyFIPS.astype(int)
    cases.countyFIPS = cases.countyFIPS.astype(int)
   
    #cases = cases[cases.countyFIPS != 0]   # these are not removed because of 'unallocated' deaths
    cases = cases.groupby(['State']).sum().reset_index()  # sum over counties
    #deaths = deaths[deaths.countyFIPS != 0]
    deaths = deaths.groupby(['State']).sum().reset_index()
    
    cases_and_deaths = pd.merge(cases, deaths, how='left', on='State')
    cases_and_deaths = cases_and_deaths.fillna(0)
    deaths_keys = [k for k in cases_and_deaths.keys() if '#Deaths' in k and not 'Unnamed' in k]
    cases_keys = [k for k in cases_and_deaths.keys() if '#Cases' in k and not 'Unnamed' in k]
    deaths = cases_and_deaths[deaths_keys].values
    cases = cases_and_deaths[cases_keys].values
    cases_and_deaths['deaths'] = [deaths[i] for i in range(deaths.shape[0])]
    cases_and_deaths['cases'] = [cases[i] for i in range(cases.shape[0])]
    cases_and_deaths['tot_deaths'] = deaths[:, -1]
    cases_and_deaths['tot_cases'] = cases[:, -1]    
    
    df = load_data.load_county_level(data_dir = '../data/')
    df['DiabetesPercentage'] = df['DiabetesPercentage'].astype(float)
    pop_density = df.groupby('StateNameAbbreviation').PopulationDensityperSqMile2010.agg(get_90_quantile).to_frame()
    pop = df.groupby('StateNameAbbreviation').PopulationEstimate2018.agg(sum).to_frame()
    median_age = df.groupby('StateNameAbbreviation').MedianAge2010.agg(get_90_quantile).to_frame()
    smoker_pct = df.groupby('StateNameAbbreviation').Smokers_Percentage.agg(get_90_quantile).to_frame()
    diabetes_pct = df.groupby('StateNameAbbreviation').DiabetesPercentage.agg(get_90_quantile).to_frame()
    hdm = df.groupby('StateNameAbbreviation').HeartDiseaseMortality.agg(get_90_quantile).to_frame()
    hospitals = df.groupby('StateNameAbbreviation')['#Hospitals'].agg(sum).to_frame()
    ICU_beds = df.groupby('StateNameAbbreviation')['#ICU_beds'].agg(sum).to_frame()
    df =  pd.concat([
                     pop_density,pop,median_age,
                     smoker_pct,diabetes_pct,hdm,
                     hospitals,ICU_beds],axis =1 )
    df['State'] = df.index
    
    state_df = pd.merge(cases_and_deaths, df, how='left', on='State')


    return state_df

In [461]:
state_df = load_state_data(dir_mod = "../data/")

In [462]:
sum(p[-1] for p in state_df['deaths'].values) ## for 4/6, this should be close to 9552

9552

In [463]:
very_important_vars = ['PopulationDensityperSqMile2010',
#                        'MedicareEnrollment,AgedTot2017',
                       #'PopulationEstimate2018',
                       #'#ICU_beds',
                       'MedianAge2010',
                       'Smokers_Percentage']
exponential = {'model_type':'exponential'}
shared_exponential = {'model_type':'shared_exponential'}
demographics = {'model_type':'shared_exponential', 'demographic_vars':very_important_vars}
linear = {'model_type':'linear'}

In [468]:
df = copy.deepcopy(state_df)

## running state-level prediction eval

In [472]:
all_methods = [exponential, 
               shared_exponential,
               #demographics,
               linear]
df = fit_and_predict.fit_and_predict_ensemble(df, 
                                              target_day=np.array(range(1, 8)),
                                              mode='eval_mode',
                                              #mode='predict_future',
                                              outcome='deaths',
                                              methods=all_methods[1:],
                                              output_key=f'predicted_deaths_ensemble_all'
                                              )
df = fit_and_predict.fit_and_predict_ensemble(df, 
                                              target_day=np.array(range(1, 8)),
                                              mode='eval_mode',
                                              #mode='predict_future',
                                              outcome='deaths',
                                              methods=[all_methods[1]],
                                              output_key=f'predicted_deaths_simple_ensemble_all'
                                              )

Feature weights
log(deaths) : 1.1822510205293864
bias : 1.3723001140188678
Feature weights
log(deaths) : 1.0759441459151247
bias : 1.3138423004688369
Feature weights
log(deaths) : 1.0437926797933805
bias : 1.278200967516959
Feature weights
log(deaths) : 1.0140128828289556
bias : 1.2512151709547663
Feature weights
log(deaths) : 0.9844898004115722
bias : 1.2360732782493025
Feature weights
log(deaths) : 0.97437289087402
bias : 1.2138380524713366
Feature weights
log(deaths) : 0.954398259323746
bias : 1.2326864386596612
Feature weights
log(deaths) : 0.9426483127024406
bias : 1.2942167202218218
--- Model Contributions ---
{'model_type': 'shared_exponential'}: 0.5023676392953276
{'model_type': 'linear'}: 0.4976323607046724
Feature weights
log(deaths) : 1.1822510205293864
bias : 1.3723001140188678
Feature weights
log(deaths) : 1.0759441459151247
bias : 1.3138423004688369
Feature weights
log(deaths) : 1.0437926797933805
bias : 1.278200967516959
Feature weights
log(deaths) : 1.0140128828289556
b

In [474]:
method_keys = [c for c in df if 'predicted' in c]
for key in method_keys:  # convert '_all' predictions to daily predictions
    if 'all' in key:
        for d in range(1, 8):
            newkey = key[:-3] + str(d)
            df[newkey] = np.array([p[d-1] for p in df[key].values])

In [453]:
df['state'] = df.State
method_keys = [c for c in df if 'predicted' in c]
preds_df = df[method_keys + ['state']]
preds_df.to_csv("../predictions/predictions_state_level_03_30.csv") # only run this if you wish to save results

In [475]:
np.sum(np.abs(df['predicted_deaths_ensemble_7'].values - np.array([p[-1] for p in df['deaths'].values])))

1853.4456776374047

In [ ]:
df['predicted_deaths_simple_ensemble_7'][:50]